# Cloud Data Transfer Speeds Benchmarking Workflow

Add overview of workflow

## Step 0: Load Required Setup Packages & Classes

Installs required workflow setup packages and calls UI generation script. If one or more of the packages don't exist in your `base` environment, they will install for you. Note that if installation is required, this cell will take a few minutes to complete execution.

In [1]:
import os
import json
import sys

print('Checking conda environment for UI depedencies...')
os.system("bash " + os.getcwd() + "/jupyter-helpers/install_ui_packages.sh")
print('All dependencies installed.')

sys.path.insert(0, os.getcwd() + '/jupyter-helpers')
import ui_helpers as ui
import pandas as pd

Checking conda environment for UI depedencies...
All dependencies installed.


## Step 1: Define Workflow Inputs

Run the following cells to generate interactive widgets allowing you to enter all workflow inputs. **All inputs must be filled out to proceed with the benchmarking process.**

### Cloud Resources

#### Compute Resources

Before defining anything else, the resources you intend to use with the benchmarking must be defined. Currently, only resources defined in the Parallel Works platform may be used. Also of note are options that will be passed to Dask: you have full control over how many cores and memory you want each worker from your cluster to use, as well as how many nodes you want to be active at a single time.

In particular, these options are included so that you can form fair comparisons between different cloud service providers (CSPs). Generally, different CSPs won't have worker nodes with the exact same specs, and in order to achieve a fair comparison between two CSPs one cluster will have to limited to not exceed to the computational power of the other.

<div class="alert alert-block alert-info">
For resources controlled from the Parallel Works platform, the <code>Resource name</code> box should be populated with the name found on the <b>RESOURCES</b>  tab.
    </div>

In [ ]:
resource = ui.resourceWidgets()
resource.display()
resources = resource.processInput()
print(f'Your resource inputs:\n {resources}')

#### Object Stores

This set of inputs is where you enter the cloud object store Universal Resource Identifiers (URIs). Both public and private buckets are supported. For the latter, ensure that you have access credentials with *at least* read, write, list, and put (copy from local storage to cloud) permissions, as format conversions will need to be made during the benchmarking process.

In [ ]:
store = ui.storageWidgets()
store.display()
storage = store.processInput()
print(f'Your storage inputs:\n {storage}')

### Datasets

#### User-Supplied Datasets

Below you can specify datasets that you want to be tested in the benchmarking. You can either enter single files or multiple files that belong to a single dataset, but that dataset match at least one of the supported formats. **Read the following input rules after running the UI cell below this one.**


1. Activate the checkbox if you desire to record your user-defined datasets. If it is not checked, none of your inputs will be recorded.

<br>

2. Input the full URI or absolute path of the data location (`<URI prefix>://bucket-name/path/to/file.extension` or `/path/to/file.extension`)
    - Use globstrings (`<URI prefix>://bucket-name/path/to/files/*` or `path/to/files/*`) to specify datasets that are split up into multiple subfiles.
    - If using a globstring, ensure that *only* files that belong to the dataset exist in that directory. The workflow will take all files in the directory before the `*` and attempt to gather them into a single dataset.
    
<br>

3. If you have a dataset stored in multiple cloud storage locations that will be used in the benchmarking, you must input the full URI of that dataset for each of these locations. That is, you must define each location of the data separately.

<br>

4. For single-file datasets > 5.4 GB (5 GiB) in size that may need to be transferred across clouds in the benchmarking, you must transfer these files to the desired locations *before* running the benchmarking.
    - This is because `gsutil`, one of the tools the benchmark uses to copy user-defined files from the original cloud storage location to other benchmarking locations, can only handle single-file transfers between CSPs that are smaller than 5 GiB.

In [ ]:
userdata = ui.userdataWidgets(storage=storage)
userdata.display()
user_files = userdata.processInput()
print(f'Your data inputs:\n {user_files}')

#### Randomly-Generated Datasets

Another option to supply data to the benchmarking is to create randomly-generated datasets. These sets can be as large as you want (as they are written in parallel), and provide a great option if you are new to the world of cloud-native data formats. There are currently two supported randomly-generated data formats: CSV and NetCDF4. Since NetCDF4 is a gridded data format, an option to specify the number of coordinate axes is also included.

In [ ]:
randgen = ui.randgenWidgets(resources=resources)
randgen.display()
randfiles = randgen.processInput()
print(f'Your randomly-generated file options:\n {randfiles}')

### TODO: Cloud-Native Format Conversion Options

**Feature not ready**

## Step 2: Notebook Setup

Executing the following cell will write all of your inputs to `inputs.json`, install miniconda3 and the "cloud-data" Python environment to all resources, and write randomly-generated files to all cloud storage locations (if any files were specified). If writing randomly-generated files, especially large ones, the execution of this cell may take a while.

<div class="alert alert-block alert-info">
While randomly-generated files are written in parallel by default, if you wish to speed up the execution of this cell, consider creating/choosing a resource with more powerful worker nodes.
    </div>

In [ ]:
print('Setting up workflow...')

user_input = json.dumps({"RESOURCES" : resources,
                         "STORAGE" : storage,
                         "USERFILES" : user_files,
                         "RANDFILES" : randfiles
                        })

with open('inputs.json', 'w') as outfile:
    outfile.write(user_input)

os.system("bash workflow_notebook_setup.sh")

print('Workflow setup complete.')

## Step 3: Run Benchmarking

### Convert File to Cloud-Native

Since one of the major goals of this benchmarking is testing legacy formats against cloud-native ones, we must convert your legacy-formatted data (CSV and NetCDF4) into their corresponding cloud-native formats. This cell will execute and time the conversion process, writing each new format in parallel. The conversion will be done using each cluster's full amount of resources, so be mindful of this feature when using clusters that are expensive to operate.

In [2]:
os.system("bash benchmarks-core/run_benchmark_step.sh \"convert-data.py\" \"conversions.csv\"")
df_conversions = pd.read_csv(os.getcwd() + '/results/csv-files/conversions.csv')
df_conversions

Waiting for worker nodes to start up...
Workers active.
Converting files in "gs://cloud-data-benchmarks" with "gcptestnew"...


Traceback (most recent call last):
  File "/home/jgreen/cloud-data-transfer-benchmarking/benchmarks-core/convert-data.py", line 181, in <module>
    ds = core.combine_nc_subfiles(base_uri, filename, storage_options, fs)
         ^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: module 'core_helpers' has no attribute 'combine_nc_subfiles'
cloud-data-transfer-benchmarking/outputs/results_tmp.csv: No such file or directory
rm: cannot remove ‘results_tmp.csv’: No such file or directory
cat: /home/jgreen/pw/storage/cloud-data-transfer-benchmarking/results/csv-files/results_tmp.csv: No such file or directory
rm: cannot remove ‘/home/jgreen/pw/storage/cloud-data-transfer-benchmarking/results/csv-files/results_tmp.csv’: No such file or directory


EmptyDataError: No columns to parse from file

### File Reads

The last computation-intensive test in the benchmarking is reading and timing files from cloud storage. This will give you an idea of what data transfer throughput you can expect when using cloud storage and different data formats in other workflows.

In [ ]:
os.system("bash benchmarks-core/run_benchmark_step.sh \"read-data.py\" \"reads.csv\"")
df_reads = pd.read_csv(os.getcwd() + '/results/csv-files/reads.csv')
df_reads

## TODO: Step 4: Visualize Results

**Feature not ready**